In [63]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,BitsAndBytesConfig,pipeline,default_data_collator
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'


In [28]:
KEYWORDS={
        #----------------------form--------------------------------------
       'firstname':['firstname','first_name','forename','first','first-name'],
       'lastname':['lastname','last_name','surename','last','last-name'] ,
       'email':['email','e-mail','username','user_name','user-name','type="email"'], 
       'password':['password'], 
       'cv':['cv','resume','curriculum','vitae'],
       'birthdate':['birthdate','birth','dateofbirth','date-of-birth','birth-date'],
       'phonenumber':['phonenumber','phone','telephone','tel'],
       'housenumber':['hausnummer'],
       'residentcountry':['resident','country'],
       'citizencountry':['citizen','citizenship','city'],
       'transcript':['transcript'],
       'coverletter':['coverletter','cover', 'letter'], 
       'picture':['picture'], 
       'address':['address','location'],
       'linkedin':['linkedin'],
       'github':['github'],
       'xing':['xing'],
       'twitter':['twitter'],
       'website':['website'],
       'postalcode':['postalcode','zip','zip-code','postal'],
       'city':['city'],
       'housenumber':['housenumber'], 
       'salary':['salary'],
       'sex':['sex','gender'], 
       'availablefrom':['noticeprieod'],
       'file':['type="file"'],
       'captcha':['captcha'],
       'submit':['apply','type="submit"'],
       'cancel':['cancel'],
       'fake':['fake'],
       'question':['question'],
       'agree':['agree'],
       'other':['other'],
       'apply':['apply'],
       'recommendation':['recommendation'],
       'workherebefore':['workherebefore'],
       'findus':['find'],
       'dropbox':['dropbox'],
       'googledrive':['google drive'],
       'workhours':['workhours'],
       'workduration':[''],
       'visa':['visa'],
       'eligible':['eligible'],
       'germanlevel':['germanlevel'],
       'company':[''],#current company
       'relocate':[''],
       'universityenrollment':[],#are a student?
       'englishlevel':[],
       #important about job
       #important about jobplace?
              
       #-------------------------------------------------------------------
       #-----------------------------------btns ---------------------------
       'login':['login','sign'],
       'username':['username'],
       'register':['register','sign'],
       'cookie':['cookie'],
       'next':['apply now','next'],
       'search':[],
       #-------------------------------------------------------------------
       
       
       
       
       }


LABEL_INDEX_TO_KEY = list(KEYWORDS.keys()) #self.personal_data.columns
i =0
LABEL_KEY_TO_INDEX = {}
for key in KEYWORDS:
            LABEL_KEY_TO_INDEX[key] =i
            i+=1 

In [ ]:
mydata = pd.read_pickle('input_text.pkl')
#mydata.reset_index(inplace=True)
mydata

In [ ]:
mydata['labels'].value_counts().plot(figsize=(10,10), kind='bar')

In [41]:
mydata[mydata['labels']=='residentcounty']#='residentcountry'
mydata.loc[45,'labels'] = 'residentcountry'

In [ ]:
"""index = (inp['input_ids']== toknizer.mask_token_id).int().argmax().item()
print(index ,o.logits.shape)
token_id = o.logits[0,index,:].argsort(descending=True)
r = toknizer.convert_ids_to_tokens(token_id[:5])
MASK = toknizer.convert_tokens_to_string(r[0:1])
"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large-mnli" , cache_dir=cache_dir)
model  = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large-mnli",
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir)

In [81]:
data = datasets.Dataset.from_pandas(mydata)
data = data.train_test_split(0.2)
train_dataset = data['train']
test_dataset = data['test']


def preprocess(example):
    #print(example['element'])
    h = [f"this text is about {i} ." for i in example['text']]
    t= tokenizer(example['element'],h,return_tensors='pt' , padding=True,truncation=True ,max_length=512).to('cuda')
    t['labels'] = [1 for i in example['labels']]
    return t
    pass

train_dataset_tokenized =  train_dataset.map(preprocess,batch_size=32,batched=True,drop_last_batch=False,remove_columns=['text', 'element', 'label', 'input_text'])
test_dataset_tokenized =  test_dataset.map(preprocess,batch_size=32,batched=True,drop_last_batch=False,remove_columns=['text', 'element', 'label', 'input_text'])

train_dataset_tokenized

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map: 100%|██████████| 121/121 [00:00<00:00, 2055.87 examples/s]


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 480
})

In [82]:
train_dataset_tokenized[0]['input_ids']

[0,
 41552,
 46797,
 1907,
 40635,
 29015,
 113,
 10,
 6374,
 12,
 179,
 42679,
 40635,
 22303,
 113,
 13561,
 40635,
 29015,
 12,
 46797,
 12,
 8569,
 113,
 10,
 6374,
 12,
 33480,
 40635,
 14869,
 35,
 50,
 90,
 6,
 37035,
 293,
 1245,
 1021,
 3624,
 618,
 459,
 4494,
 9791,
 113,
 766,
 40635,
 462,
 113,
 7241,
 43779,
 43651,
 40635,
 1529,
 113,
 45729,
 40635,
 2723,
 6,
 37035,
 293,
 1245,
 1021,
 3624,
 2968,
 329,
 113,
 923,
 40635,
 254,
 32373,
 225,
 113,
 1380,
 40635,
 48408,
 12,
 1409,
 922,
 298,
 506,
 364,
 134,
 267,
 45328,
 288,
 118,
 246,
 46479,
 2,
 2,
 9226,
 2788,
 16,
 59,
 21,
 41552,
 46797,
 1907,
 40635,
 29015,
 113,
 10,
 6374,
 12,
 179,
 42679,
 40635,
 22303,
 113,
 13561,
 40635,
 29015,
 12,
 46797,
 12,
 12196,
 113,
 10,
 6374,
 12,
 33480,
 40635,
 7325,
 35,
 633,
 90,
 405,
 523,
 6,
 1690,
 1725,
 605,
 2723,
 242,
 1021,
 3624,
 542,
 1334,
 858,
 298,
 2262,
 113,
 766,
 40635,
 1343,
 113,
 7241,
 43779,
 43651,
 40635,
 1529,
 113,
 

In [97]:
import torchmetrics
import numpy as np
from var_dump import var_dump
def compute_metric(inp):
    preds , label_ids  = inp
    p_label = np.argmax(preds,1)
    
    metric = torchmetrics.Accuracy('multiclass',num_classes=3)
    return metric(p_label , label_ids )

trainer =  Trainer(
    model=model,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metric,
    
    args= TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1
        
        
        
    )
    
)

trainer.evaluate()


AttributeError: 'numpy.ndarray' object has no attribute 'is_floating_point'

In [ ]:
"""bid = json.loads(d['actions'][1]['pos_candidates'][0]['attributes'])['backend_node_id']
print(d['actions'][1]['pos_candidates'])
soup = BeautifulSoup(lll['html'][1])#d['actions'][1]['raw_html'])
#soup = BeautifulSoup(d['actions'][3]['raw_html'])
#s = soup.find( '*' )
# Find all 'div' tags
soup = soup.find('form')
all_tags = soup.find_all()

# Remove all tags except for 'link', 'input', and 'select' tags
i=0
for element in all_tags:
    if element.name  in ['div','text']: 
    #if element.name not in ['a','link', 'input', 'select','radio','button']:
        
        element.unwrap()
    else:
        element['backend_node_id']=i
        
        if 'class' in element.attrs:
            del element['class']
            pass
        if 'bounding_box_rect' in element.attrs:
            del element['bounding_box_rect']
            pass
        if 'is_clickable' in element.attrs:
            del element['is_clickable']
            pass
        
            
        i+=1

     

s = soup.prettify()
#b = s.split("/>")
x = toknizer(s,return_tensors='pt')
print(x['input_ids'].shape[1],x['input_ids'].shape[1]/16348)


print(s)
if False:
    for i in b:
        print(i)
        print('---------------------')
"""

In [ ]:
from IPython.display import display, HTML
display(HTML(s))


In [ ]:
display(HTML(s))


In [ ]:
mydata.iloc[1,:]